In [239]:
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
route = gpd.read_file("Tramroute.shp")

In [240]:
stops = gpd.read_file("Tramstops.shp")

In [241]:
stops.rename({'STOP_ID':'id'},inplace=True)
stops['id'] = stops['id'].apply(int)

In [242]:
stops.to_file("Tramstops.shp")

In [243]:
stops.head()

,id,name,LATITUDE,LONGITUDE,TICKETZONE,TRAM_TRACK,ROUTES_USI,geometry
0,18224,20-Kings Way/Sturt St (Southbank),-37.830136,144.965310,1,1230,1,POINT (144.96535 -37.83015)
1,18225,19-Miles St/Sturt St (Southbank),-37.828431,144.965755,1,1229,1,POINT (144.96585 -37.82846)
2,18226,18-Grant St/Sturt St (Southbank),-37.825779,144.966634,1,1228,1,POINT (144.96666 -37.82579)
3,18227,17-Southbank Bvd/Sturt St (Southbank),-37.823771,144.967235,1,1227,1,POINT (144.96727 -37.82378)
4,18481,22-Dorcas St/Eastern Rd (South Melbourne),-37.832221,144.964992,1,1232,1,POINT (144.96504 -37.83220)


In [244]:
line = route.geometry[0]
# First coords of line (start + end)
coords = [l for l in line.coords]
pointcoords = []
# Add the coords from the points
for i in range(len(stops)):
    pointcoords += stops.geometry[i].coords
    coords += stops.geometry[i].coords
# Calculate the distance along the line for each point
dists = [line.project(Point(p)) for p in coords]
pointdists = [line.project(Point(p)) for p in pointcoords]
# sort the coordinates
sortpointcoords = [p for (d, p) in sorted(zip(pointdists, pointcoords))]

coords = [p for (d, p) in sorted(zip(dists, coords))]
lines = [LineString([coords[i], coords[i+1]]) for i in range(len(coords)-1)]

In [251]:
stops['distance'] = stops.to_crs({'init': 'epsg:28355'})['geometry'].apply(lambda p: route.to_crs({'init': 'epsg:28355'}).geometry[0].project(p))
stops.sort_values(['distance'],inplace=True)
stops.reset_index(drop=True,inplace=True)

In [252]:
stops.loc[0,'distance'] = 0
stops.loc[stops.index[-1],'distance'] = route.to_crs({'init': 'epsg:28355'}).geometry[0].length

In [253]:
stops['distance'] = stops['distance'].diff()

In [265]:
stops.crs

{'init': 'epsg:4326'}

In [257]:
result = []
sublist = []

separators = sortpointcoords[1:-1]

for x in coords:
    if x in separators:
        if sublist:
            sublist.append(x)
            result.append(LineString(sublist))
        sublist = [x]
    else:
        sublist.append(x)
result.append(LineString(sublist))

[<shapely.geometry.linestring.LineString object at 0x7ff3a9c05cd0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c050d0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c05fa0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c05ca0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c05550>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c052b0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e4c0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e430>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c05970>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e460>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e850>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e4f0>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e490>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e520>, <shapely.geometry.linestring.LineString object at 0x7ff3a9c2e

In [282]:
newshape = gpd.GeoDataFrame(geometry = result,crs="EPSG:4326")
newshape['route_dist'] = stops['distance'].shift(-1)
newshape['stop1'] = stops['id']
newshape['stop1N'] = stops['name']
newshape['stop2'] = stops['id'].shift(-1)
newshape['stop2'] = newshape['stop2'].apply(int)
newshape['stop2N'] = stops['name'].shift(-1)
newshape.index.rename('id',inplace=True)
newshape.reset_index(inplace=True)
newshape['peak_speed'] = 20
newshape['free_speed'] = 40
newshape['routes'] = '0'

newshape.head()

,id,geometry,route_dist,stop1,stop1N,stop2,stop2N,peak_speed,free_speed,routes
0,0,"LINESTRING (144.97803 -37.74280, 144.97803 -37...",175.945821,19368,135-Bell St/Nicholson St (Coburg),18730,134-Merribell Ave/Nicholson St (Coburg),20,40,0
1,1,"LINESTRING (144.97781 -37.74437, 144.97772 -37...",227.069166,18730,134-Merribell Ave/Nicholson St (Coburg),18729,133-Harding St/Nicholson St (Coburg),20,40,0
2,2,"LINESTRING (144.97739 -37.74638, 144.97690 -37...",340.509932,18729,133-Harding St/Nicholson St (Coburg),18728,132-Crozier St/Nicholson St (Coburg),20,40,0
3,3,"LINESTRING (144.97689 -37.74943, 144.97660 -37...",215.520283,18728,132-Crozier St/Nicholson St (Coburg),18727,131-Rennie St/Nicholson St (Coburg),20,40,0
4,4,"LINESTRING (144.97658 -37.75135, 144.97636 -37...",254.149004,18727,131-Rennie St/Nicholson St (Coburg),18726,130-The Avenue/Nicholson St (Coburg),20,40,0


In [283]:
newshape.crs

'EPSG:4326'

In [280]:
newshape.to_file("Tramsegments.shp")

In [348]:
import pandas as pd

from simpledbf import Dbf5

dbf = Dbf5('PTLINK.DBF')

df = dbf.to_dataframe()

In [311]:
distsum = 0
timesum = 0
def runloop(row):
    global distsum, timesum
    distsum += row['DIST']
    timesum += row['TIME']
    if row['STOPB'] == 1:
        totaldist = distsum
        totaltime = timesum
        distsum = 0
        timesum = 0
        return [totaldist, totaltime]

In [314]:
df[['linkdist','linktime']] = df.apply(lambda row: pd.Series(runloop(row)), axis=1)

<ipython-input-314-5f9aefdd6579>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['distsum','timesum']] = df.apply(lambda row: pd.Series(runloop(row)), axis=1)


In [319]:
df = df.drop(['distsum','timesum'], axis=1)

In [322]:
df = df.loc[(df.STOPA != 0) | (df.STOPB != 0)]

In [333]:
df.columns

Index(['PERIOD', 'PERIODN', 'A', 'B', 'MODE', 'OPERATOR', 'LINENO', 'NAME',
       'LONGNAME', 'VEHTYPE', 'VEHNAME', 'SEATCAP', 'CRUSHCAP', 'ADJCAP',
       'FARESYS', 'LINKSEQ', 'DIST', 'TIME', 'HEADWAY', 'SERVICE', 'NVEH',
       'VOL', 'TOTSCAP', 'TOTCCAP', 'TOTACAP', 'PT_VC', 'STOPA', 'STOPB',
       'ONA', 'OFFA', 'ONB', 'OFFB', 'TRAIN', 'TRAM', 'BUS', 'VLINE', 'COACH',
       'linkdist', 'linktime'],
      dtype='object')

In [324]:
df['STARTNODE'] = df['A'].shift(1)

In [335]:
df = df.loc[df['STOPB'] == 1]

In [336]:
df['A'] = df['STARTNODE'].apply(int)

KeyError: 'STARTNODE'

In [330]:
df.drop('STARTNODE', axis=1, inplace=True)

In [340]:
df = df[['PERIOD','A','B','MODE','LINENO','NAME','LONGNAME','HEADWAY','SERVICE','linkdist','linktime']]

In [349]:
df.groupby(['LONGNAME']).size()

LONGNAME
=St Kilda-Glen Waverley    580
AIRCRAFT - FOOTSCRAY       200
AIRPORT TO FRANKSTON       484
AIRPORT TO MT ELIZA        242
AIRPORT TO ROSEBUD         121
                          ... 
Yarraville - Newport       560
new route 2385             152
new route 2422              92
new route 385              152
new route 422               92
Length: 1105, dtype: int64